# 手动实现经典NLP模型算法——LSTM

In [1]:
from dataset.ptb import load_data
from lstm.gated_rnnlm import GatedRnnlm
from common.optimizer import SGD
from common.utils import RnnlmTrainer, eval_perplexity

## Ⅰ、加载PTB语料数据集

In [2]:
corpus, word_to_id, id_to_word = load_data("train")
corpus_val, _, _ = load_data('val')
corpus_test, _, _ = load_data('test')
corpus_size = len(corpus)
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]
ts = corpus[1:]
data_size = len(xs)
'corpus size: %d, vocabulary size: %d' % (corpus_size, vocab_size)

'corpus size: 929589, vocabulary size: 10000'

## Ⅱ、创建LSTM语言模型

In [3]:
# 定义超参数
batch_size = 20
wordvec_size = 650
hidden_size = 650
time_size = 35
lr = 5.0
max_epoch = 40
max_grad = 0.25
dropout = 0.5

In [4]:
# 定义模型
model = GatedRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

## Ⅲ、模型训练

In [5]:
best_ppl = float('inf')
for epoch in range(max_epoch):
  trainer.fit(xs, ts, max_epoch=1, batch_size=batch_size, time_size=time_size, max_grad=max_grad)

  model.reset_state()
  ppl = eval_perplexity(model, corpus_val)
  print('valid perplexity: ', ppl)

  if best_ppl > ppl:
    best_ppl = ppl
    model.save_params()
  else:
    lr /= 4.0
    optimizer.lr = lr

  model.reset_state()
  print('-' * 50)

| epoch 1 |  iter 1 / 1327 | time 1[s] | perplexity 9990.28
| epoch 21 |  iter 21 / 1327 | time 47[s] | perplexity 2980.99
| epoch 41 |  iter 41 / 1327 | time 88[s] | perplexity 1386.60
| epoch 61 |  iter 61 / 1327 | time 128[s] | perplexity 1089.28
| epoch 81 |  iter 81 / 1327 | time 165[s] | perplexity 997.82
| epoch 101 |  iter 101 / 1327 | time 208[s] | perplexity 920.03
| epoch 121 |  iter 121 / 1327 | time 252[s] | perplexity 925.47
| epoch 141 |  iter 141 / 1327 | time 306[s] | perplexity 885.39


KeyboardInterrupt: 

## Ⅳ、模型评估

In [ ]:
model.reset_state()
model.load_params("GatedRnnlm.pkl")
ppl_test = eval_perplexity(model, corpus_test)
print('test perplexity: ', ppl_test)